# Emotion Detection using CNN (Clean Notebook)
This notebook contains clean and ready-to-run code for training a CNN-based Emotion Detection model and launching a Gradio app.

In [ ]:
!pip install tensorflow gradio opencv-python numpy

## Upload and Extract Dataset ZIP

In [ ]:
from google.colab import files
import zipfile, os

uploaded = files.upload()
zip_filename = list(uploaded.keys())[0]

extract_path = '/content/dataset'
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

os.listdir(extract_path)

## Load Dataset

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

BASE_DIR = '/content/dataset'

train_dir, test_dir = None, None
for f in os.listdir(BASE_DIR):
    if 'train' in f.lower(): train_dir = os.path.join(BASE_DIR, f)
    if 'test' in f.lower(): test_dir = os.path.join(BASE_DIR, f)

IMG_SIZE = (48, 48)
BATCH_SIZE = 32

train_gen = ImageDataGenerator(rescale=1/255)
test_gen = ImageDataGenerator(rescale=1/255)

train_data = train_gen.flow_from_directory(train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')
test_data = test_gen.flow_from_directory(test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')

num_classes = len(train_data.class_indices)

## Train CNN Model

In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(48,48,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_data, validation_data=test_data, epochs=5)

## Save Model

In [ ]:
model.save('/content/emotion_model.h5')
print('Model saved!')

## Run Gradio App

In [ ]:
import gradio as gr
import numpy as np
import cv2

labels = list(train_data.class_indices.keys())

def predict_emotion(img):
    img_resized = cv2.resize(img, (48,48))
    img_norm = img_resized / 255.0
    img_input = np.expand_dims(img_norm, axis=0)
    preds = model.predict(img_input)[0]
    return {labels[i]: float(preds[i]) for i in range(len(labels))}

demo = gr.Interface(fn=predict_emotion, inputs=gr.Image(type='numpy'), outputs=gr.Label(num_top_classes=len(labels)))
demo.launch()